<h1>Scraping fidelity.com</h1>
In this assignment, you will scrape data from fidelity.com. The goal of the exercise is to get the latest sector performance data from the US markets, and to get the total market capitalization for each sector. 

The end result is to write a function: <i>get_us_sector_performance()</i> that will return a list of tuples. Each tuple should correspond to a sector and should contain the following data:
<li>the sector name
<li>the amount the sector has moved
<li>the market capitalization of the sector
<li>the market weight of the sector
<li>a link to the fidelity page for that sector

<p>
The data should be sorted by decreasing order of market weight. I.e., the sector with the highest weight should be in the first tuple, etc.

In [2]:
def f(x):
      yield x+1
      yield x+2
g=f(9)

for v in g:

     print(v)

10
11


<h3>Process</h3>
<li>Get a list of sectors and the links to the sector detail pages from the url (see function)
<li>Loop through the list and call the function <i>get_sector_change_and_market_cap(sector_page_link)</i> for each sector
<li>Accumulate the name, the change, the capitalization, the weight and the link for each sector in output_list (see function)
<li>Sort the list by market weight

<b>Notes:</b>
<li>Note that the market weight is a string with a % sign at the back. You will need to get rid of the % and convert the string into a float before you can sort it
<li>Your starting data is the url listed below. You need to extract all data, including links to the sector pages, from the page at this url
<li>To sort a list of tuples by an arbitrary element, use the example at the bottom of this notebook

In [146]:
def get_sector_change_and_market_cap(sector_page_link):    
    import requests
    from bs4 import BeautifulSoup
    response=requests.get(sector_page_link)
    if not response.status_code == 200:
        return 0
    results_page=BeautifulSoup(response.content,'lxml')
    spans=results_page.find_all('span')
    try:
        sector_change=float(spans[45].get_text().strip('%'))
        sector_market_cap=spans[47].get_text()
        sector_market_weight=float(spans[49].get_text().strip('%'))
        return sector_change,sector_market_cap,sector_market_weight
    except:
        return 0
    
    
def get_us_sector_performance():
    import requests
    from bs4 import BeautifulSoup
    output_list = list()
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"
    response=requests.get(url)
    if not response.status_code == 200:
        return output_list
    try:
        results_page=BeautifulSoup(response.content,'lxml')
        sectors=results_page.find('table',{'class':'sector-list'}).find_all('div',{'class':'heading'})
        for sector in sectors:
            sector_name=sector.find('a').get_text()
            sector_link="https://eresearch.fidelity.com"+sector.find('a').get('href')
            output_list.append((sector_name,get_sector_change_and_market_cap(sector_link)[0],get_sector_change_and_market_cap(sector_link)[1],get_sector_change_and_market_cap(sector_link)[2],sector_link))
            output_list.sort(key=itemgetter(3)) 
        return output_list
    except:
        return output_list
    

    

In [147]:
get_us_sector_performance()

[('Materials',
  0.65,
  '$1.82T',
  2.72,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15'),
 ('Real Estate',
  0.48,
  '$1.39T',
  3.23,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),
 ('Utilities',
  0.15,
  '$1.49T',
  3.61,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),
 ('Energy',
  -0.47,
  '$3.10T',
  4.57,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),
 ('Consumer Staples',
  0.68,
  '$3.92T',
  7.6,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'),
 ('Industrials',
  0.34,
  '$4.12T',
  9.37,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('Consumer Discretionary',
  0.81,
  '$5.23